In [ ]:
!pip install yfinance # https://pypi.org/project/yfinance/
!pip install hurst # https://pypi.org/project/hurst/

In [ ]:
import yfinance as yf
from hurst import compute_Hc
import pandas as pd
import numpy as np
import datetime

In [ ]:
# volume weighted average price

def VWAP(window, price, volume):

  VWAP_array = np.array([])

  for k0 in range(window,price.shape[0]):
    value = sum(price[k0-window:k0] * volume[k0-window:k0]) / sum(volume[k0-window:k0])
    VWAP_array = np.append(VWAP_array,value)

  VWAP_array = np.append([np.NaN] * window, VWAP_array)

  return VWAP_array


# Hurst exponent

def hurst_it(series, window, return_type):

  error_flag = False
  H_array = np.array([])
  original_length = series.shape[0]
  series = series.dropna()

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    try:
      H, c, dataset = compute_Hc(slice, kind='price', simplified=True)
      H_array = np.append(H_array,H)
    except:
      H_array = np.append(H_array,np.NaN)
      error_flag = True

  new_length = len(H_array)
  diff = original_length - new_length
  if error_flag == False:
    H_array = np.append(H_array[0]*np.ones(diff), H_array)
  elif error_flag == True:
    H_array = np.append([np.NaN] * diff, H_array)

  mean_line = np.nanmean(H_array, axis=0) * np.ones(len(H_array))

  if return_type == 'hurst':
    return H_array
  elif return_type == 'mean':
    return mean_line


# exponential moving average

def EMA(lookback, series):
  alpha = 2 / (lookback +1)
  ewm = series.ewm(alpha = alpha, adjust = False).mean()
  return ewm


# Fractal Adaptive Moving Average

# http://www.stockspotter.com/Files/frama.pdf
# https://medium.com/geekculture/fractal-volatility-bands-new-trading-horizons-66ee445be198

def frama(price,FDI):

  frama_array = np.array([])
  frama_array = np.append(frama_array,price[0])

  for k in range(1,price.shape[0]):
    A = np.exp(-4.6 * (FDI[k] - 1))
    A = min(max(A,0.01),1)
    value = A * price[k] + (1 - A) * frama_array[k-1]
    frama_array = np.append(frama_array,value)

  return frama_array


# custom realised volatility

def volatility(series, window):

  std_array = np.array([])

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    std = slice.std()
    std_array = np.append(std_array,std)

  std_array = np.append(std_array[0]*np.ones(window),std_array)
  return std_array


# volatility banding

def bands(frama_series,price_series,window,std_multiple,return_type):

  a = pd.DataFrame(std_multiple*volatility(price_series,window))
  b = frama_series
  a.index = b.index

  if return_type == 'lower':
    return b.subtract(a[0], fill_value=0)
  elif return_type == 'upper':
    return b.add(a[0], fill_value=0)


# geometric mean

def geo_mean_overflow(iterable):
    a = np.log(iterable)
    return np.exp(a.mean())


In [ ]:
temp_15min = []
temp_60min = []
temp_daily = []

data_combined_15m = pd.DataFrame()
data_combined_60m = pd.DataFrame()
data_combined_daily = pd.DataFrame()

intervals = ['15m','60m','1d']    # '15m' = 60 days , '60m' = 730 days , '1d' = unlimited
now = datetime.datetime.now()
end = str(now.year)+"-"+str(now.month)+"-"+str(now.day)

VWAP_window = 7
std_multiple = 2
nrows = 1000

symbols = ['SPY','QQQ','IWM','DUFN.SW','CGC','SAVE','LVS','CPRI','PLTR','EXPE','MTB','BKNG','GDRX','NUH.AX','MOLN.SW']

In [ ]:
symbols_dict_url = 'https://raw.githubusercontent.com/jefferygao1984/METIS_IDS_PROJECT/main/symbols_dict_02.csv'
symbols_dict = pd.read_csv(symbols_dict_url)
symbols_dict['LABEL'] = symbols_dict['TICKER_TYPE'] + ' : ' + symbols_dict['TICKER']
# symbols_dict = symbols_dict.tail(5)
symbols_dict

,TICKER,SOURCE,TICKER_TYPE,DESCRIPTION,QUAD_PROB_MASK,QUAD1_GOLDILOCKS,QUAD2_REFLATION,QUAD3_STAGFLATION,QUAD4_DEFLATION,LABEL
0,XLV,YAHOO,ETF,Health Care Select Sector SPDR Fund,1,0,0,0,1,ETF : XLV
1,XLP,YAHOO,ETF,SPDR S&P - Consumer Staples,1,0,0,0,1,ETF : XLP
2,XLU,YAHOO,ETF,SPDR S&P - Utilities,1,0,0,1,1,ETF : XLU
3,XLK,YAHOO,ETF,SPDR S&P - Technology,1,1,1,1,0,ETF : XLK
4,XLY,YAHOO,ETF,SPDR S&P - Consumer Discretionary,1,1,1,0,0,ETF : XLY
5,XLB,YAHOO,ETF,SPDR S&P - Materials,1,1,0,0,0,ETF : XLB
6,XLI,YAHOO,ETF,SPDR S&P - Industrials,1,1,1,0,0,ETF : XLI
7,XLE,YAHOO,ETF,SPDR S&P - Energy,1,0,1,1,0,ETF : XLE
8,XLF,YAHOO,ETF,SPDR S&P - Financials,1,1,1,0,0,ETF : XLF
9,XOP,YAHOO,ETF,SPDR S&P - Oil & Gas,1,1,0,0,0,ETF : XOP


In [ ]:
t0 = datetime.datetime.now()

for interval in intervals:
    for ticker in symbols_dict['TICKER']:
    #for ticker in symbols:

        current_time = datetime.datetime.now()
        current_time = str(current_time)

        if interval == '15m':
            start = now - datetime.timedelta(days=30)    # max = 60 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        elif interval == '60m':
            start = now - datetime.timedelta(days=90)    # max = 730 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        elif interval == '1d':
            # start = '1900-01-01'
            start = now - datetime.timedelta(days=365*2)
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)

        temp_df = yf.download(ticker, start=start, end=end, interval=interval, prepost = False, threads = True, proxy = None)
        temp_df = temp_df.tail(nrows)

        if interval == '15m':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = min(60,int(0.25*temp_df.shape[0]))
        elif interval == '60m':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = min(60,int(0.25*temp_df.shape[0]))
        elif interval == '1d':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = min(60,int(0.25*temp_df.shape[0]))

        # cleansing + preprocessing
        temp_df['Volume'] = temp_df['Volume'].replace(0, 1)
        temp_df['Volume'] = temp_df['Volume'].fillna(1)
        temp_df = temp_df.fillna(method='ffill')
        temp_df = temp_df.fillna(method='bfill')
        temp_df['OHLC'] = 0.25 * (temp_df['Open'] + temp_df['High'] + temp_df['Low'] + temp_df['Close'])
        # calc VWAP
        price_series = temp_df['OHLC'].copy()
        volume_series = temp_df['Volume'].copy()
        VWAP_array = VWAP(VWAP_window, price_series, volume_series)
        temp_df['VWAP'] = [x for x in VWAP_array]
        temp_df = temp_df.dropna()
        # calc Hurst
        price_series = temp_df['VWAP'].copy()
        temp_hurst = hurst_it(price_series, hurst_window, 'hurst')
        temp_df['HURST'] = [x for x in temp_hurst]
        # calc Fractal Dimension Index
        temp_df['FDI'] = [2 - x for x in temp_hurst]
        temp_df['FDI_MU'] = np.nanmean(temp_df['FDI']) * np.ones(temp_df['FDI'].shape[0])
        # calc mean Hurst
        temp_hurst = hurst_it(price_series, hurst_window, 'mean')
        temp_df['HURST_MU'] = [x for x in temp_hurst]
        # calc Exponential Moving Average
        ewm = EMA(ewm_lookback, price_series)
        temp_df['EMA'] = [x for x in ewm]
        # calc Fractal Adaptive Moving Average
        temp_df = temp_df.dropna()
        temp_frama = frama(temp_df.VWAP,temp_df.FDI)
        temp_df['FRAMA'] = [x for x in temp_frama]
        # calculate volatility bands
        temp_vola = bands(temp_df['FRAMA'],temp_df['VWAP'],vola_band_window,std_multiple,'lower')
        temp_df['LOWER'] = [x for x in temp_vola]
        temp_vola = bands(temp_df['FRAMA'],temp_df['VWAP'],vola_band_window,std_multiple,'upper')
        temp_df['UPPER'] = [x for x in temp_vola]
        # assign labels
        temp_df['TICKER'] = ticker
        temp_df['LABEL'] = ticker

        if interval == '15m':
            frames = [data_combined_15m, temp_df]
            data_combined_15m = pd.concat(frames)
            temp_15min.append(temp_df.shape[0])
        elif interval == '60m':
            frames = [data_combined_60m, temp_df]
            data_combined_60m = pd.concat(frames)
            temp_60min.append(temp_df.shape[0])
        else:
            frames = [data_combined_daily, temp_df]
            data_combined_daily = pd.concat(frames)
            temp_daily.append(temp_df.shape[0])

data_combined_15m.columns = data_combined_15m.columns.str.replace(' ', '_')
data_combined_60m.columns = data_combined_60m.columns.str.replace(' ', '_')
data_combined_daily.columns = data_combined_daily.columns.str.replace(' ', '_')

data_combined_15m.reset_index(inplace=True)
data_combined_60m.reset_index(inplace=True)
data_combined_daily.reset_index(inplace=True)

data_combined_15m['idx'] = data_combined_15m['Datetime']
data_combined_60m['idx'] = data_combined_60m['Datetime']
data_combined_daily['idx'] = data_combined_daily['Date']
data_combined_15m = data_combined_15m.set_index('idx')
data_combined_60m = data_combined_60m.set_index('idx')
data_combined_daily = data_combined_daily.set_index('idx')

temp = data_combined_15m[['VWAP','TICKER']].copy()
data_combined_15m_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_VWAP_refactored = data_combined_15m_VWAP_refactored.fillna(method='bfill')
data_combined_15m_VWAP_refactored = data_combined_15m_VWAP_refactored.fillna(method='ffill')

temp = data_combined_60m[['VWAP','TICKER']].copy()
data_combined_60m_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_VWAP_refactored = data_combined_60m_VWAP_refactored.fillna(method='bfill')
data_combined_60m_VWAP_refactored = data_combined_60m_VWAP_refactored.fillna(method='ffill')

temp = data_combined_daily[['VWAP','TICKER']].copy()
data_combined_daily_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_VWAP_refactored = data_combined_daily_VWAP_refactored.fillna(method='bfill')
data_combined_daily_VWAP_refactored = data_combined_daily_VWAP_refactored.fillna(method='ffill')

t0 = datetime.datetime.now() - t0
print('yfinance exec time: ',t0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# symbols = ['SPY','QQQ','IWM','DUFN.SW','CGC','SAVE','LVS','CPRI','PLTR','EXPE','MTB','BKNG','GDRX','NUH.AX','MOLN.SW']
interval = '1d'    # 15m , 60m , 1d
ticker = 'GDRX'

if interval == '15m':
  subset = data_combined_15m[data_combined_15m['TICKER'] == ticker]
elif interval == '60m':
  subset = data_combined_60m[data_combined_60m['TICKER'] == ticker]
elif interval == '1d':
  subset = data_combined_daily[data_combined_daily['TICKER'] == ticker]

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    subplot_titles=('', '', ''),
                    row_width=[0.2, 0.2, 0.6], vertical_spacing=0.01,
                    specs=[[{"secondary_y": True}],[{"secondary_y": False}],[{"secondary_y": False}]])

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['VWAP'],
    mode='lines+markers',
    name='VWAP',
    line=dict(color='blue',width=1),
    marker=dict(color='blue',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['LOWER'],
    mode='lines+markers',
    name='LOWER',
    line=dict(color='green',width=1),
    marker=dict(color='green',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['UPPER'],
    mode='lines+markers',
    name='UPPER',
    line=dict(color='red',width=1),
    marker=dict(color='red',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FRAMA'],
    mode='lines+markers',
    name='FRAMA',
    line=dict(color='black',width=1),
    marker=dict(color='black',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Bar(
    x=subset.index,
    y=subset['Volume'],
    name='VOLUME',
    marker_color = 'purple',
    marker_line_color='purple',
    marker_line_width=1.0
), row=1, col=1, secondary_y=True,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['HURST'],
    mode='lines+markers',
    name='HURST',
    line=dict(color='burlywood',width=1),
    marker=dict(color='burlywood',size=2)
), row=2, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['HURST_MU'],
    mode='lines+markers',
    name='MEAN HURST',
    line=dict(color='orange',width=1),
    marker=dict(color='orange',size=2)
), row=2, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FDI'],
    mode='lines+markers',
    name='FDI',
    line=dict(color='darkseagreen',width=1),
    marker=dict(color='darkseagreen',size=2)
), row=3, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FDI_MU'],
    mode='lines+markers',
    name='MEAN FDI',
    line=dict(color='mediumturquoise',width=1),
    marker=dict(color='mediumturquoise',size=2)
), row=3, col=1, secondary_y=False,)

description = ticker

epsilon = 0.05
geomean_window = 20

if interval == '15m':
  hurst_tail = data_combined_15m[data_combined_15m.TICKER==ticker].HURST.iloc[-1]
  iter = data_combined_15m[data_combined_15m.TICKER==ticker].VWAP.tail(geomean_window)
elif interval == '60m':
  hurst_tail = data_combined_60m[data_combined_60m.TICKER==ticker].HURST.iloc[-1]
  iter = data_combined_60m[data_combined_60m.TICKER==ticker].VWAP.tail(geomean_window)
elif interval == '1d':
  hurst_tail = data_combined_daily[data_combined_daily.TICKER==ticker].HURST.iloc[-1]
  iter = data_combined_daily[data_combined_daily.TICKER==ticker].VWAP.tail(geomean_window)

iter = 1 + iter.pct_change()
GEOMEAN = round(geo_mean_overflow(iter) - 1,6)

if hurst_tail < 0.5 - epsilon:
  TREND = 'MEAN REVERTING'
elif hurst_tail > 0.5 + epsilon:
  if GEOMEAN <= 0:
    TREND = 'BEARISH'
  else:
    TREND = 'BULLISH'
else: 
  TREND = 'BROWNIAN'

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', height=800, width=1300, title_text= interval + ': '+ description + ' (' + TREND + ')')

#max_y_1_plot_sec = 5 * subset['Volume'].max()
#min_y_1_plot_pri = subset['LOWER'].median() - 1.1*(subset['LOWER'].max() - subset['LOWER'].median())
#max_y_1_plot_pri = subset['UPPER'].median() + 1.1*(subset['UPPER'].max() - subset['UPPER'].median())

max_y_1_plot_sec = 6 * subset['Volume'].max()
min_y_1_plot_pri = 1 * subset['LOWER'].min()
max_y_1_plot_pri = subset['UPPER'].max()

fig.update_xaxes(row=1, col=1, showgrid=False, type='category', showticklabels=False)
fig.update_yaxes(row=1, col=1, range=[0, max_y_1_plot_sec], secondary_y=True, showgrid=False)
fig.update_yaxes(row=1, col=1, range=[min_y_1_plot_pri, max_y_1_plot_pri], secondary_y=False, showgrid=False)
fig.update_xaxes(row=2, col=1, showgrid=False, type='category', showticklabels=False)
fig.update_xaxes(row=3, col=1, showgrid=False, type='category', showticklabels=False)

fig.show()